In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
from onvif import ONVIFCamera
from pynput import keyboard

In [2]:
!which python
!find /home/donfa1con/.virtualenvs/python2.7/ -name onvif -type d
!find /home/donfa1con/.virtualenvs/python2.7/ -name wsdl -type d

/home/donfa1con/.virtualenvs/python2.7/bin/python
/home/donfa1con/.virtualenvs/python2.7/lib/python2.7/site-packages/onvif
/home/donfa1con/.virtualenvs/python2.7/wsdl


In [4]:
PATH_TO_WSDL = '/home/donfa1con/.virtualenvs/python2.7/wsdl/'
login = 'diziganshin'
pwd = 'JtIfa2n6bgCrqmG9'
url = '192.168.15.42'
port = 80

In [5]:
class Camera(object):
    def __init__(self, url, port, login, pwd, PATH_TO_WSDL):
        self.camera = ONVIFCamera(url, port, login, pwd, PATH_TO_WSDL)
        self.ptz = self.camera.create_ptz_service()
        self.media = self.camera.create_media_service()
        self.imaging = self.camera.create_imaging_service()
        self.media_profile_token = self.media.GetProfiles()[0]._token
        self.imaging_video_source_token = self.media.GetVideoSources()[0]._token
        self.media_profile_ptz_configuration_token = \
            self.media.GetProfiles()[0].PTZConfiguration._token
        self.requests = {}
        
    def init(self):
        self.init_ptz_request('GetStatus')
        self.init_ptz_request('AbsoluteMove')
        self.init_ptz_request('ContinuousMove')
        self.init_ptz_config_request()
        
        self.init_imaging_request('Move')
        self.init_imaging_request('SetImagingSettings')
        self.init_imaging_request('GetMoveOptions')
        
        
    def init_ptz_request(self, name):
        self.requests[name] = self.ptz.create_type(name)
        self.requests[name].ProfileToken = self.media_profile_token
        
    def init_imaging_request(self, name):
        self.requests[name] = self.imaging.create_type(name)
        self.requests[name].VideoSourceToken = self.imaging_video_source_token
        
    def init_ptz_config_request(self):
        name = 'GetConfigurationOptions'
        self.requests[name] = self.ptz.create_type(name)
        self.requests[name].ConfigurationToken = \
            self.media_profile_ptz_configuration_token
        
    def get_position_status(self):
        name = 'GetStatus'
        status = self.ptz.GetStatus(self.requests[name])
        x = status.Position.PanTilt._x
        y = status.Position.PanTilt._y
        zoom = status.Position.Zoom._x
        return {'x': x, 'y': y, 'zoom': zoom}
    
    def get_range_continuous_move(self):
        name = 'GetConfigurationOptions'
        ptz_config = self.ptz.GetConfigurationOptions(self.requests[name])
        spaces = ptz_config.Spaces
        XMAX = spaces.ContinuousPanTiltVelocitySpace[0].XRange.Max
        XMIN = spaces.ContinuousPanTiltVelocitySpace[0].XRange.Min
        YMAX = spaces.ContinuousPanTiltVelocitySpace[0].YRange.Max
        YMIN = spaces.ContinuousPanTiltVelocitySpace[0].YRange.Min
        ZMAX = spaces.ContinuousZoomVelocitySpace[0].XRange.Max
        ZMIN = spaces.ContinuousZoomVelocitySpace[0].XRange.Min
        return {'x': [XMAX, XMIN], 'y': [YMAX, YMIN], 'zoom': [ZMAX, ZMIN]}
    
    def get_range_absolute_move(self):
        name = 'GetConfigurationOptions'
        ptz_config = self.ptz.GetConfigurationOptions(self.requests[name])
        spaces = ptz_config.Spaces
        XMAX = spaces.AbsolutePanTiltPositionSpace[0].XRange.Max
        XMIN = spaces.AbsolutePanTiltPositionSpace[0].XRange.Min
        YMAX = spaces.AbsolutePanTiltPositionSpace[0].YRange.Max
        YMIN = spaces.AbsolutePanTiltPositionSpace[0].YRange.Min
        ZMAX = spaces.AbsoluteZoomPositionSpace[0].XRange.Max
        ZMIN = spaces.AbsoluteZoomPositionSpace[0].XRange.Min
        return {'x': [XMAX, XMIN], 'y': [YMAX, YMIN], 'zoom': [ZMAX, ZMIN]}
    
    def get_focus_move_options(self):
        name = 'GetMoveOptions'
        focus_config = self.imaging.GetMoveOptions(self.requests[name])
        return focus_config
    
    def set_absolute_position(self, x_position=0, y_position=0, zoom_position=0):
        name = 'AbsoluteMove'
        self.requests[name].Position.PanTilt._x = x_position
        self.requests[name].Position.PanTilt._y = y_position
        self.requests[name].Position.Zoom._x = zoom_position
    
    def make_absolute_move(self):
        name = 'AbsoluteMove'
        self.ptz.AbsoluteMove(self.requests[name])
        self.ptz.Stop({'ProfileToken': self.requests[name].ProfileToken})
        
    def set_continuous_velocity(self, x_velocity=0, y_velocity=0, zoom_velocity=0):
        name = 'ContinuousMove'
        self.requests[name].Velocity.PanTilt._x = x_velocity
        self.requests[name].Velocity.PanTilt._y = y_velocity
        self.requests[name].Velocity.Zoom._x = zoom_velocity
        
    def make_continuous_move(self):
        name = 'ContinuousMove'
        self.ptz.ContinuousMove(self.requests[name])
        self.ptz.Stop({'ProfileToken': self.requests[name].ProfileToken})
        
    def set_manual_focus(self):
        name = "SetImagingSettings"
        request = {'VideoSourceToken': self.imaging_video_source_token,
                   'ImagingSettings': {'Focus': {'AutoFocusMode': 'MANUAL'}}
                  }
        self.imaging.SetImagingSettings(request)
        
    def reset_manual_focus(self):
        name = "SetImagingSettings"
        request = {'VideoSourceToken': self.imaging_video_source_token,
                   'ImagingSettings': {'Focus': {'AutoFocusMode': 'AUTO'}}
                  }
        self.imaging.SetImagingSettings(request)
        
    def move_focus(self, speed=0):
        name = "Move"
        self.imaging.Move({'VideoSourceToken': self.imaging_video_source_token,
            'Focus': {
            'Continuous': {
                'Speed': speed
            }
        }})
        self.imaging.Stop({'VideoSourceToken': self.imaging_video_source_token})

In [6]:
camera = Camera(url, port, login, pwd, PATH_TO_WSDL)

In [7]:
camera.init()

### 1. Absolute move: диагностика и перемещение 

In [8]:
camera.get_range_absolute_move()

{'x': [1.0, -1.0], 'y': [1.0, -1.0], 'zoom': [1.0, 0.0]}

In [9]:
camera.get_range_continuous_move()

{'x': [1.0, -1.0], 'y': [1.0, -1.0], 'zoom': [1.0, -1.0]}

In [10]:
camera.get_position_status()

{'x': 0.152444, 'y': -0.549111, 'zoom': 0.0}

In [11]:
camera.set_absolute_position(0., -0.7, 0)
camera.make_absolute_move()

In [12]:
camera.get_position_status()

{'x': 0.0, 'y': -0.7, 'zoom': 0.0}

In [13]:
camera.set_absolute_position(0.2, -0.4, 1)
camera.make_absolute_move()

In [14]:
camera.get_position_status()

{'x': 0.150333, 'y': -0.4, 'zoom': 1.0}

In [15]:
del camera

### 2. Программа перемещения камеры в Continuous move

In [16]:
class KeyboardCamera(Camera):
    def __init__(self, delta, *args):
        super(KeyboardCamera, self).__init__(*args)
        self._delta = delta
        self.delta = delta
        
    def left_up(self):
        self.set_continuous_velocity(-self.delta, self.delta, 0)
        self.make_continuous_move()
        
    def up(self):
        self.set_continuous_velocity(0, self.delta, 0)
        self.make_continuous_move()
        
    def right_up(self):
        self.set_continuous_velocity(self.delta, self.delta, 0)
        self.make_continuous_move()
        
    def left(self):
        self.set_continuous_velocity(-self.delta, 0, 0)
        self.make_continuous_move()
        
    def right(self):
        self.set_continuous_velocity(self.delta, 0, 0)
        self.make_continuous_move()
        
    def left_down(self):
        self.set_continuous_velocity(-self.delta, -self.delta, 0)
        self.make_continuous_move()
        
    def down(self):
        self.set_continuous_velocity(0, -self.delta, 0)
        self.make_continuous_move()
        
    def right_down(self):
        self.set_continuous_velocity(self.delta, -self.delta, 0)
        self.make_continuous_move()
        
    def zoom_plus(self):
        self.set_continuous_velocity(0, 0, self.delta)
        self.make_continuous_move()
        
    def zoom_plus(self):
        self.set_continuous_velocity(0, 0, -self.delta)
        self.make_continuous_move()
        
    def delta_plus(self):
        self.delta = min(self.delta + self._delta, 1)
    
    def delta_minus(self):
        self.delta = max(self.delta - self._delta, -1)
            
    def on_press_with_continous_move(self, key):
        try:
            if key.char == '+':
                self.zoom_plus()
            if key.char == '-':
                self.zoom_minus()
            if key.char == '/':
                self.delta_plus()
            if key.char == '*':
                self.delta_minus()
        except AttributeError:
            if key == keyboard.Key.home:
                self.left_up()
            if key == keyboard.Key.up:
                self.up()
            if key == keyboard.Key.page_up:
                self.right_up()
            if key == keyboard.Key.left:
                self.left()
            if key == keyboard.Key.right:
                self.right()
            if key == keyboard.Key.end:
                self.left_down()
            if key == keyboard.Key.down:
                self.down()
            if key == keyboard.Key.page_down:
                self.right_down()

    
    def on_press(self, key):
        self.on_press_with_continous_move(key)
        
    def on_release(self, key):
        # Stop listener
        if key == keyboard.Key.esc:
            return False

    def listen_keyboard(self):
        with keyboard.Listener(on_press=self.on_press,
                               on_release=self.on_release) as listener:
            listener.join()

In [17]:
delta = 5./20
keyboard_camera = KeyboardCamera(delta, url, port, login, pwd, PATH_TO_WSDL)

In [18]:
keyboard_camera.init()

In [19]:
keyboard_camera.listen_keyboard()

### 3. Программа ручного фокуса

In [21]:
keyboard_camera.get_focus_move_options()

(MoveOptions20){
   Continuous = 
      (ContinuousFocusOptions){
         Speed = 
            (FloatRange){
               Min = -7.0
               Max = 7.0
            }
      }
 }

In [23]:
keyboard_camera.set_manual_focus()
keyboard_camera.move_focus(speed=-7)
keyboard_camera.move_focus(speed=7)
keyboard_camera.reset_manual_focus()